# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
!pip3 install gmaps
!jupyter nbextension enable --py gmaps

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
read_csv = "output_data/cities.csv"
city_data = pd.read_csv(read_csv)

#  Get the indices of cities that have humidity over 100%.
humidity = city_data.loc[city_data["Humidity"] > 100]
indexes = []
if humidity.index.any():
    indexes = humidity.index
    city_data = city_data.drop(index=indexes)

# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
clean_city_data = pd.DataFrame(city_data)
del clean_city_data["Unnamed: 0"]
clean_city_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cambuí,-22.6122,-46.0575,59.50,67,56,3.87,BR,1627941561
1,Longyearbyen,78.2186,15.6401,42.24,91,100,9.91,SJ,1627941561
2,Bambous Virieux,-20.3428,57.7575,59.25,88,20,8.05,MU,1627941562
3,Talnakh,69.4865,88.3972,66.29,53,100,4.59,RU,1627941562
4,Pevek,69.7008,170.3133,35.04,81,100,19.08,RU,1627941563
...,...,...,...,...,...,...,...,...,...
576,Praia da Vitória,38.7333,-27.0667,75.76,88,75,6.55,PT,1627941535
577,Visby,57.6409,18.2960,60.85,82,40,10.36,SE,1627941830
578,Wellington,-41.2866,174.7756,54.82,79,75,5.01,NZ,1627941809
579,Talcahuano,-36.7167,-73.1167,56.82,70,47,10.92,CL,1627941615


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = clean_city_data[["Lat", "Lng"]]
weight = clean_city_data["Humidity"]
max_humidity = weight.max()

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
nice_temp = (clean_city_data.loc[(clean_city_data["Max Temp"]>70) & (clean_city_data["Max Temp"]<80)])
nice_wind = nice_temp.loc[nice_temp["Wind Speed"]<10]
new_data = nice_wind.loc[nice_wind["Cloudiness"]==0]
# new_data = new_data.reset_index()
new_data = pd.DataFrame(new_data)
# del new_data["index"]
new_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,Nador,35.1740,-2.9287,71.85,78,0,3.44,MA,1627941593
86,Jamestown,42.0970,-79.2353,73.09,61,0,3.24,US,1627941595
149,Kerman,30.2832,57.0788,75.06,17,0,0.00,IR,1627941463
152,Hambantota,6.1241,81.1185,79.14,94,0,9.22,LK,1627941634
168,Neijiang,29.5835,105.0622,77.67,84,0,1.39,CN,1627941645
186,Vila Velha,-20.3297,-40.2925,73.96,78,0,1.14,BR,1627941499
245,Dzaoudzi,-12.7887,45.2699,71.49,83,0,3.44,YT,1627941681
361,Seoul,37.5683,126.9778,77.76,85,0,1.90,KR,1627941645
439,Neyshabur,36.2133,58.7958,74.82,16,0,3.22,IR,1627941775
533,Aksay,47.2600,39.8700,73.62,32,0,3.53,RU,1627941812


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = new_data[["City","Country","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""
params = {
    "radius": 5000,
    "types":"lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    adrress = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"] = adrress["results"][0]["name"]
    except:
        print("No hotel found")
hotel_df


,City,Country,Lat,Lng,Hotel Name
81,Nador,MA,35.1740,-2.9287,Hôtel La Giralda
86,Jamestown,US,42.0970,-79.2353,Hampton Inn & Suites Jamestown
149,Kerman,IR,30.2832,57.0788,هتل آپارتمان آسام
152,Hambantota,LK,6.1241,81.1185,Bungalow 63
168,Neijiang,CN,29.5835,105.0622,Han'an Hotel
186,Vila Velha,BR,-20.3297,-40.2925,Quality Suites Vila Velha
245,Dzaoudzi,YT,-12.7887,45.2699,Le Rocher
361,Seoul,KR,37.5683,126.9778,롯데호텔 서울
439,Neyshabur,IR,36.2133,58.7958,Amiran Hotel
533,Aksay,RU,47.2600,39.8700,Chernaya Zhemchuzhina


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer on top of heat map

# Create heat layer
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))